# Bonus Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. Use Ironhack's database to load the pokemon data (db: pokemon, table: pokemon_stats). 

In [2]:
# Import libraries
import pandas as pd

In [34]:
# Load the data:
pokemon = pd.read_csv('../data/pokemon.csv')
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [39]:
# Your code here

type1 = pokemon['Type 1'].unique()
type2 = pokemon['Type 2'].unique()
unique_types = np.concatenate((type1, type2))
unique_types = list(set(unique_types))

len(unique_types) # you should see 19
unique_types

[nan,
 'Grass',
 'Water',
 'Fairy',
 'Ice',
 'Psychic',
 'Dragon',
 'Bug',
 'Electric',
 'Fire',
 'Ghost',
 'Rock',
 'Flying',
 'Fighting',
 'Poison',
 'Normal',
 'Ground',
 'Dark',
 'Steel']

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [51]:
# pokemon_totals = []

for unique in unique_types:
    if isinstance(unique, str):
        pokemon_totals.append(pokemon[(pokemon['Type 1'] == unique) | (pokemon['Type 2'] == unique)]['Total'])
        
pokemon_totals

[0      318
 1      405
 2      525
 3      625
 48     320
       ... 
 783    335
 784    494
 785    494
 786    494
 787    494
 Name: Total, Length: 95, dtype: int64, 9      314
 10     405
 11     530
 12     630
 59     320
       ... 
 759    500
 760    320
 762    330
 763    500
 799    600
 Name: Total, Length: 126, dtype: int64, 40     323
 41     483
 44     270
 45     435
 131    460
 187    218
 188    210
 189    245
 190    405
 198    250
 199    420
 225    300
 226    450
 303    198
 304    278
 305    518
 306    618
 322    190
 328    380
 329    480
 366    590
 487    310
 519    545
 591    545
 606    280
 607    480
 737    303
 738    371
 739    552
 752    341
 753    462
 754    341
 755    480
 770    525
 772    431
 773    500
 777    470
 792    680
 795    600
 796    700
 Name: Total, dtype: int64, 94     475
 98     525
 133    455
 142    535
 156    580
 233    430
 238    250
 239    450
 243    330
 257    305
 395    300
 396    480
 397  

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [54]:
from scipy import stats

stats.f_oneway(*pokemon_totals)

F_onewayResult(statistic=6.617538296005533, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?

pvalue < 0.5 --> podemos rejeitar a hipótese nula. Por isso as médias do 'Total' de cada tipo de pokemon são diferentes.